In [40]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [104]:
df = pd.read_csv("Egypt_Houses_Price.csv")

In [105]:
df

,Type,Price,Bedrooms,Bathrooms,Area,Furnished,Level,Compound,Payment_Option,Delivery_Date,Delivery_Term,City
0,Duplex,4000000,3.0,3.0,400.0,No,7,Unknown,Cash,Ready to move,Finished,Nasr City
1,Apartment,4000000,3.0,3.0,160.0,No,10+,Unknown,Cash,Ready to move,Finished,Camp Caesar
2,Apartment,2250000,3.0,2.0,165.0,No,1,Unknown,Cash,Ready to move,Finished,Smoha
3,Apartment,1900000,3.0,2.0,230.0,No,10,Unknown,Cash,Ready to move,Finished,Nasr City
4,Apartment,5800000,2.0,3.0,160.0,No,Ground,Eastown,Cash,Ready to move,Semi Finished,New Cairo - El Tagamoa
...,...,...,...,...,...,...,...,...,...,...,...,...
27356,Town House,890000,3.0,2.0,240.0,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Unknown,North Coast
27357,Town House,4000000,4.0,3.0,218.0,Unknown,Unknown,Unknown,Cash or Installment,Unknown,Finished,New Cairo - El Tagamoa
27358,Twin House,13800000,3.0,4.0,308.0,No,Unknown,Cairo Festival City,Cash,Ready to move,Semi Finished,New Cairo - El Tagamoa
27359,Stand Alone Villa,35000000,4.0,4.0,478.0,Unknown,Unknown,Unknown,Unknown Payment,Unknown,Finished,Mokattam


In [106]:
def drop_na(df):
    return df.dropna()

In [107]:
def drop_10_plus_bedrooms(df):
    return df[df['Bedrooms'] != '10+']

In [108]:
def drop_unknown_price(df):
    return df[df['Price'] != 'Unknown']

In [109]:
def convert_data_types(df):
    return df.astype({'Price': 'float', 'Bedrooms': 'float', 'Bathrooms': 'float', 'Area': 'float'})

In [110]:
def drop_unnecessary_columns(df):
    return df.drop(['Compound','Delivery_Term','Delivery_Date'],axis='columns')

In [111]:
def type_cleaning_and_map(df):
    # Convert 'Type' column to lowercase
    df['Type'] = df['Type'].apply(lambda x: x.lower())
    
    # Replace 'stand alone villa' with 'standalone villa'
    df['Type'] = df['Type'].str.replace('stand alone villa', 'standalone villa')
    
    # Drop rows where 'Type' is 'penthouse' or 'studio'
    df.drop(df[(df['Type'] == 'penthouse') | (df['Type'] == 'studio')].index, inplace=True)
    
    # Define type mapping
    type_mapping = {'apartment': 0, 'duplex': 1, 'chalet': 2, 'twin house': 3, 'town house': 4, 'standalone villa': 5}
    
    # Map 'Type' values to numeric values using the defined mapping
    df['Type'] = df['Type'].map(type_mapping)
    
    return df

In [112]:
def level_cleaning(df):
    #dropping chalets with unknown level
    df.drop(df[(df['Level'] == 'Unknown') & (df['Type'] == 2)].index,inplace=True)
    
    #making villas have level -1
    
    for type in df.Type.unique():
        if type not in [0,1,2]:
            df.loc[df['Type'] == type, 'Level'] = '-1'
    
    #dropping duplexs with unknown level
    df.drop(df[(df['Level'] == 'Unknown') & (df['Type'] == 1)].index,inplace=True)

    #dropping +10 or highest levels 
    df.drop(df[(df['Level'] == '10+') | (df['Level'] == 'Highest')].index,inplace=True)
    
    #rplacing ground level to be 0
    df['Level'] = df['Level'].str.replace("Ground",'0')

    #making the level integer
    df['Level'] = df['Level'].astype(int)

    return df

In [113]:
def city_cleaning_and_encoding(df):
    # Count the occurrences of each city
    city_counts = df['City'].value_counts()
    
    # Filter cities with counts less than or equal to 10
    city_counts_less_than_10 = city_counts[city_counts <= 10]
    
    # Replace rare cities with 'other'
    df['City'] = df['City'].apply(lambda x: 'other' if x in city_counts_less_than_10 else x)
    
    # Drop rows with a specific value in the 'City' column
    df.drop(df[df['City'] == "(View phone number)"].index, inplace=True)
    
    # Perform one-hot encoding for the 'City' column
    dummies = pd.get_dummies(df['City'])
    dummies = dummies.astype(int)
    
    # Concatenate the one-hot encoded columns with the original DataFrame
    df = pd.concat([df, dummies.drop('other', axis='columns')], axis='columns')
    
    # Drop the original 'City' column
    df.drop(['City'], axis='columns', inplace=True)
    
    return df

In [114]:
def furnished_cleaning_and_map(df):
    df['Furnished'] = df['Furnished'].str.replace('Unknown','No')
    furnished_mapping = {'No' : 0 , 'Yes' : 1}
    df['Furnished'] = df['Furnished'].map(furnished_mapping)
    return df


In [115]:
def map_payment_option(df):
    df['Payment_Option'] = df['Payment_Option'].str.replace('Unknown Payment','Cash or Installment')
    payment_mapping = {'Cash': 0, 'Installment': 1, 'Cash or Installment': 2}
    df['Payment_Option'] = df['Payment_Option'].map(payment_mapping)
    return df

In [116]:
class CustomOutlierRemover(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Removing outliers based on specified conditions
        X.drop(X[X.Area / X.Bedrooms < 8].index, inplace=True)
        X.drop(X[X.Bathrooms > X.Bedrooms + 2].index, inplace=True)
        X = X[~((X['Price'] < 800000) | (X['Price'] >= 65000000))]
        
        # Calculate price per square meter
        X['price_per_m2'] = X['Price'] / X['Area']
        
        # Removing price per square meter outliers for each property type
        df_out = pd.DataFrame()
        for key, subdf in X.groupby('Type'):
            m = np.mean(subdf.price_per_m2)
            st = np.std(subdf.price_per_m2)
            reduced_df = subdf[(subdf.price_per_m2 >= (m - st)) & (subdf.price_per_m2 <= (m + st))]
            df_out = pd.concat([df_out, reduced_df], ignore_index=True)


        # Drop the price_per_m2 column
        df_out.drop(columns=['price_per_m2'], inplace=True)
        
        return df_out

In [119]:
def add_logPrice_and_drop_price(df):
    df['Price'] = np.log(df['Price'])
    return df

In [121]:
pipeline = Pipeline(steps=[
    ('drop_na', FunctionTransformer(drop_na, validate=False)),
    ('drop_10_plus_bedrooms', FunctionTransformer(drop_10_plus_bedrooms, validate=False)),
    ('drop_unknown_price', FunctionTransformer(drop_unknown_price, validate=False)),
    ('convert_data_types', FunctionTransformer(convert_data_types, validate=False)),
    ('drop_unnecessary_columns', FunctionTransformer(drop_unnecessary_columns, validate=False)),
    ('type_cleaning_and_map', FunctionTransformer(type_cleaning_and_map, validate=False)),
    ('level_cleaning', FunctionTransformer(level_cleaning, validate=False)),
    ('city_cleaning_and_encoding', FunctionTransformer(city_cleaning_and_encoding, validate=False)),
    ('furnished_cleaning_and_map', FunctionTransformer(furnished_cleaning_and_map, validate=False)),
    ('map_payment_option', FunctionTransformer(map_payment_option, validate=False)),
    ('outlier_remover', CustomOutlierRemover()),
    ('add_logPrice_and_drop_price', FunctionTransformer(add_logPrice_and_drop_price, validate=False)),
])

In [122]:
df = pipeline.fit_transform(df)

C:\Users\dell\AppData\Local\Temp\ipykernel_7544\3250556869.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['price_per_m2'] = X['Price'] / X['Area']


In [123]:
df

,Type,Price,Bedrooms,Bathrooms,Area,Furnished,Level,Payment_Option,10th of Ramadan,15 May City,...,Shorouk City,Shubra,Sidi Beshr,Smoha,Tanta,West Somid,Zagazig,Zahraa Al Maadi,Zamalek,Zezenia
0,0,14.626441,3.0,2.0,165.0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,14.457364,3.0,2.0,230.0,0,10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,14.427936,4.0,3.0,222.0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,14.316286,2.0,2.0,144.0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,14.260196,3.0,3.0,200.0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16304,5,15.520259,5.0,4.0,364.0,0,-1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
16305,5,16.296242,4.0,6.0,392.0,0,-1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16306,5,16.618871,4.0,4.0,620.0,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16307,5,16.588099,5.0,5.0,530.0,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
x = df.drop(['Price'],axis='columns')
y = df['Price']

In [125]:
x

,Type,Bedrooms,Bathrooms,Area,Furnished,Level,Payment_Option,10th of Ramadan,15 May City,6th of October,...,Shorouk City,Shubra,Sidi Beshr,Smoha,Tanta,West Somid,Zagazig,Zahraa Al Maadi,Zamalek,Zezenia
0,0,3.0,2.0,165.0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,3.0,2.0,230.0,0,10,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,4.0,3.0,222.0,0,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2.0,2.0,144.0,0,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,3.0,3.0,200.0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16304,5,5.0,4.0,364.0,0,-1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16305,5,4.0,6.0,392.0,0,-1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16306,5,4.0,4.0,620.0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16307,5,5.0,5.0,530.0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
X, X_test, Y, y_test = train_test_split(x,y,test_size=0.2,random_state=10)

In [127]:
X_train, X_val, y_train, y_val = train_test_split(X,Y,test_size=0.2,random_state=10)

In [ ]:
xgb_regressor = XGBRegressor(max_depth = 16 , max_leaves= 16)

In [128]:
xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=16, max_leaves=16,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [129]:
xgb_regressor.score(X_train,y_train)

0.8672651001836169

In [131]:
xgb_regressor.score(X_val,y_val)

0.8523592678739095

In [132]:
xgb_regressor.score(X_test,y_test)

0.8480636265114512